This module provides utilities to  download IGS station data files
(e.g., observation and navigation files) from the specified FTP server

In [1]:
#import modules
import os
import shutil
import urllib.request
from datetime import datetime, timedelta
from contextlib import closing
from urllib.request import URLError

In [2]:
__all__ = ["gps_week", "igs_station_data","download_igs_product"]

def gps_week(day_of_year, year):
    """
    Convert day of year and year to GPS week.

    Parameters
    ----------
    day_of_year : int
        Day of the year (1-365 or 1-366 for leap years).
    year : int
        The year (e.g., 2023).

    Returns
    -------
    int
        The corresponding GPS week number.
    """
    gps_epoch = datetime(1980, 1, 6)

    # Convert year and day of year to a date
    target_date = datetime(year, 1, 1) + timedelta(days=day_of_year - 1)

    # Calculate difference in days from GPS epoch
    days_since_epoch = (target_date - gps_epoch).days

    # Calculate GPS week
    return days_since_epoch // 7

def igs_station_data(station_name, doy_i, doy_f, year, country_name="IND", save_path=""):
    """
    Download IGS station data (observation and navigation files) for a given station and date range.

    Parameters
    ----------
    station_name : str
        Name of the GNSS station (e.g., "IISC").
    doy_i : int
        Start day of the year (DOY).
    doy_f : int
        End day of the year (DOY).
    year : int
        The year of the data (e.g., 2023).
    country_name : str, optional
        Country code (e.g., "IND" for India). Default is "IND".
    save_path : str, optional
        Directory to save the downloaded files. Default is the current directory.

    Returns
    -------
    None
        Downloads the files to the specified directory.
    """
    base_url = r"ftp://gssc.esa.int/gnss/data/daily/"

    for i in range(doy_i, doy_f + 1):  # Include `doy_f`
        day_of_year = f"{i:03}"  # Zero-padded day of year
        path = f"{base_url}{year}/{day_of_year}/"

        # File names
        file_MO = f"{station_name}00{country_name}_R_{year}{day_of_year}0000_01D_30S_MO.crx.sum.gz"
        file_MN = f"{station_name}00{country_name}_R_{year}{day_of_year}0000_01D_MN.rnx.gz"

        # Download observation file
        try:
            with closing(urllib.request.urlopen(path + file_MO)) as r:
                save_path1 = os.path.join(save_path, file_MO) if save_path else file_MO
                with open(save_path1, 'wb') as f:
                    shutil.copyfileobj(r, f)
                    print(f"Observation file for DOY {i} has been downloaded")
        except URLError:
            print(f"Observation file for DOY {i} is missing")

        # Download navigation file
        try:
            with closing(urllib.request.urlopen(path + file_MN)) as r:
                save_path1 = os.path.join(save_path, file_MN) if save_path else file_MN
                with open(save_path1, 'wb') as f:
                    shutil.copyfileobj(r, f)
                    print(f"Navigation file for DOY {i} has been downloaded")
        except URLError:
            print(f"Navigation file for DOY {i} is missing")


def download_igs_product(year, doy_i, doy_f, product_name, save_path=""):
    """
    Downloads a specified IGS rapid product (SP3, CLK, ERP) for a range of days in a given year.

    Parameters
    ----------
    year : int
        Year for which the data is downloaded.
    doy_i : int
        Starting day of the year (1-365/366).
    doy_f : int
        Ending day of the year (1-365/366).
    product_name : str
        Name of the product to download ("SP3", "CLK", or "ERP").
    save_dir : str, optional
        Directory where the downloaded files will be saved. Default is "IGS_Products".
    path : str, optional
        Base FTP path to IGS products. Default is "ftp://gssc.esa.int/gnss/products/".
    """

    path = "ftp://gssc.esa.int/gnss/products/"
    # Validate product name
    valid_products = {
        "SP3": "IGS0OPSRAP_{year}{doy:03d}0000_01D_15M_ORB.SP3.gz",
        "CLK": "IGS0OPSRAP_{year}{doy:03d}0000_01D_05M_CLK.CLK.gz",
        "ERP": "IGS0OPSRAP_{year}{doy:03d}0000_01D_01D_ERP.ERP.gz"
    }
    if product_name not in valid_products:
        raise ValueError(f"Invalid product name: {product_name}. Valid options are {list(valid_products.keys())}")

    # Create the directory if it doesn't exist
    os.makedirs(save_path, exist_ok=True)

    # Download the specified product
    for doy in range(doy_i,doy_f + 1):
        week = gps_week(doy, year)
        product_path = f"{week}/" + valid_products[product_name].format(year=year, doy=doy)

        try:
            with closing(urllib.request.urlopen(path + product_path)) as r:
                local_file = os.path.join(save_path, os.path.basename(product_path))
                with open(local_file, 'wb') as f:
                    shutil.copyfileobj(r, f)
            print(f"Downloaded {product_name} file for day {doy}: {local_file}")
        except URLError:
            print(f"{product_name} file for day {doy} is missing")

        print(f"Completed downloads for day of year {doy}")

**Example usage of igs_station_data function**

In [3]:

# Example usage of igs_station_data function
if __name__ == "__main__":


    # Parameters
    station_name = "IISC"        # Station name (e.g., "IISC")
    doy_i = 152                  # Start day of the year (DOY)
    doy_f = 156                  # End day of the year (DOY)
    year = 2023                  # Year of the data
    country_name = "IND"         # Country code (e.g., "IND" for India)
    save_path = r"D:\TA\CE334\test"  # Directory to save the files (ensure it exists)

    # Create the directory if it does not exist
    os.makedirs(save_path, exist_ok=True)

    # Call the function
    ##IGS Station Data
    igs_station_data(
        station_name=station_name,
        doy_i=doy_i,
        doy_f=doy_f,
        year=year,
        country_name=country_name,
        save_path=save_path
    )
    #Product
    product_name = "SP3"  # Specify the product name here (e.g., "SP3", "CLK", "ERP")
    download_igs_product(year, doy_i, doy_f, product_name,save_path)

Observation file for DOY 152 has been downloaded
Navigation file for DOY 152 has been downloaded
Observation file for DOY 153 has been downloaded
Navigation file for DOY 153 has been downloaded
Observation file for DOY 154 has been downloaded
Navigation file for DOY 154 has been downloaded
Observation file for DOY 155 has been downloaded
Navigation file for DOY 155 has been downloaded
Observation file for DOY 156 has been downloaded
Navigation file for DOY 156 has been downloaded
Downloaded SP3 file for day 152: D:\TA\CE334\test/IGS0OPSRAP_20231520000_01D_15M_ORB.SP3.gz
Completed downloads for day of year 152
Downloaded SP3 file for day 153: D:\TA\CE334\test/IGS0OPSRAP_20231530000_01D_15M_ORB.SP3.gz
Completed downloads for day of year 153
Downloaded SP3 file for day 154: D:\TA\CE334\test/IGS0OPSRAP_20231540000_01D_15M_ORB.SP3.gz
Completed downloads for day of year 154
Downloaded SP3 file for day 155: D:\TA\CE334\test/IGS0OPSRAP_20231550000_01D_15M_ORB.SP3.gz
Completed downloads for day